# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Deterministic LangGraph Agent with Plan-Execute

**Description:** Ship a production-grade LangGraph agent: deterministic plan-execute, strict JSON schemas, thread memory, SQLite checkpoints, and a single FastAPI /agent endpoint.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Why This Approach Works

Building production-ready GenAI agents means solving three hard problems: **unpredictable behavior**, **runaway costs**, and **crash recovery**. Most agents fail in production because they rely on non-deterministic LLM calls, lack structured validation, and can't resume after failures.

This tutorial shows you how to build a **deterministic, resumable agent** using LangGraph's state machine, Pydantic's strict schemas, and SQLite checkpoints. You'll deploy a single FastAPI endpoint that plans, executes, and synthesizes answers—reliably.

**What you'll build:**
- A deterministic LangGraph state machine with plan → execute → finalize flow
- Strict Pydantic schemas for all inputs, outputs, and state
- SQLite-backed checkpoints for crash recovery
- A FastAPI `/agent` endpoint with typed responses
- Executable test flows demonstrating resume-on-failure

**Prerequisites:**
- Python 3.10+
- Basic FastAPI, LangChain, and LangGraph knowledge
- OpenAI API key

**Why these tools:**
- **LangGraph over custom FSMs:** Native checkpoints + typed state out of the box
- **Pydantic over dicts:** Strict validation + type safety at every boundary
- **FastAPI over Flask:** Async-first + OpenAPI schema generation
- **SQLite checkpoints:** Zero-config persistence; upgrade to Postgres for multi-worker production

All code runs in Google Colab—copy-paste cells to execute end-to-end.

---

## How It Works (High-Level Overview)

**Data flow:**
1. FastAPI receives `POST /agent` with `thread_id` and `user_input`
2. LangGraph invokes the state machine:
   - **plan** node: LLM generates a validated plan (max 5 steps, allowed tools only)
   - **execute** node: Runs each step with retries, validates I/O, logs results
   - **finalize** node: Synthesizes final answer + updates memory
   - **error** node: Captures failures as structured `ErrorEnvelope`
3. SQLite checkpointer persists state after each node
4. Response returns `FinalResult` (200) or `ErrorEnvelope` (422/500)

**State keys:**
- `thread_id`: Conversation identifier
- `user_input`: Current query
- `memory`: Conversation summary + timestamp
- `plan`: Validated plan with steps
- `step_index`: Current execution position
- `results`: Tool outputs
- `final_result` / `error`: Terminal states

**Determinism guarantees:**
- `temperature=0` + `seed=42` for LLM
- Strict Pydantic schemas forbid extra fields
- Retries with exponential backoff + timeouts
- Checkpoints enable exact resume from last completed node

---

## Setup & Installation

Run this in a Colab notebook or local environment:

In [ ]:
!pip install langgraph==0.2.28 langchain-core==0.3.15 langchain-openai==0.2.1 fastapi==0.115.0 pydantic==2.9.2 uvicorn==0.32.0 python-dotenv==1.0.1 nest-asyncio==1.6.0

**Create project structure:**

In [ ]:
!mkdir -p app
!touch app/__init__.py

**Set your OpenAI key** (use Colab secrets or `.env`):

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_MODEL"] = "gpt-4o-mini"

---

## Step-by-Step Implementation

### Step 1: Configuration

**Why:** Centralize environment variables and validate API key at startup to fail fast.

In [ ]:
%%writefile app/config.py
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY is required")

---

### Step 2: Schemas

**Why:** Strict Pydantic models enforce deterministic I/O and prevent schema drift.

In [ ]:
%%writefile app/schemas.py
from typing import List, Dict, Any, Literal
from pydantic import BaseModel, Field, StrictStr, StrictInt, StrictFloat, ConfigDict

AllowedTool = Literal["search", "retrieve_doc", "calculator"]

class PlanStep(BaseModel):
    tool: AllowedTool
    params: Dict[str, Any]
    model_config = ConfigDict(extra="forbid")

class Plan(BaseModel):
    title: StrictStr
    rationale: StrictStr
    steps: List[PlanStep] = Field(min_items=1, max_items=5)  # Cap at 5 steps
    model_config = ConfigDict(extra="forbid")

class FinalResult(BaseModel):
    answer: StrictStr
    evidence: List[Dict[str, Any]]
    plan_summary: StrictStr
    metadata: Dict[str, Any] = Field(default_factory=dict)  # Avoid mutable default
    model_config = ConfigDict(extra="forbid")

class Memory(BaseModel):
    summary: StrictStr = ""
    last_updated_ts: float = 0.0
    model_config = ConfigDict(extra="forbid")

class ErrorEnvelope(BaseModel):
    code: StrictStr
    message: StrictStr
    details: Dict[str, Any] = Field(default_factory=dict)
    partial_results: List[Dict[str, Any]] = Field(default_factory=list)
    model_config = ConfigDict(extra="forbid")

class AgentState(BaseModel):
    thread_id: str
    user_input: str
    memory: Dict[str, Any] = Field(default_factory=dict)
    plan: Dict[str, Any] = Field(default_factory=dict)
    step_index: int = 0
    results: List[Dict[str, Any]] = Field(default_factory=list)
    final_result: Dict[str, Any] = Field(default_factory=dict)
    error: Dict[str, Any] = Field(default_factory=dict)
    model_config = ConfigDict(extra="allow")  # LangGraph adds internal keys

---

### Step 3: Tools

**Why:** Validated tools with retries, timeouts, and logging ensure reliable execution.

In [ ]:
%%writefile app/tools.py
import asyncio
import time
import logging
from typing import Dict, Any
from pydantic import BaseModel, StrictStr, StrictInt, StrictFloat, ValidationError, ConfigDict, Field

logger = logging.getLogger("agent.tools")
logger.setLevel(logging.INFO)

# Tool input/output schemas
class SearchInput(BaseModel):
    query: StrictStr
    top_k: StrictInt = 3
    model_config = ConfigDict(extra="forbid")

class SearchOutput(BaseModel):
    results: List[Dict[str, StrictStr]]

class RetrieveDocInput(BaseModel):
    doc_id: StrictStr
    model_config = ConfigDict(extra="forbid")

class RetrieveDocOutput(BaseModel):
    content: StrictStr
    source: StrictStr

class CalculatorInput(BaseModel):
    expression: StrictStr
    model_config = ConfigDict(extra="forbid")

class CalculatorOutput(BaseModel):
    result: StrictFloat

# In-memory corpus
DOCS = {
    "doc:langgraph": {
        "title": "LangGraph Overview",
        "content": "LangGraph lets you build stateful, deterministic agents with graphs and checkpointers.",
        "source": "https://langchain-ai.github.io/langgraph/"
    },
    "doc:pydantic": {
        "title": "Pydantic",
        "content": "Pydantic provides type hints and data validation using Python type annotations.",
        "source": "https://docs.pydantic.dev/latest/"
    },
}

async def tool_search(inp: SearchInput) -> SearchOutput:
    q = inp.query.lower()
    hits = [{"doc_id": k, "title": v["title"]} for k, v in DOCS.items() if q in v["title"].lower() or q in v["content"].lower()]
    return SearchOutput(results=hits[:inp.top_k])

async def tool_retrieve_doc(inp: RetrieveDocInput) -> RetrieveDocOutput:
    d = DOCS.get(inp.doc_id)
    if not d:
        raise ValueError(f"doc_id not found: {inp.doc_id}")
    return RetrieveDocOutput(content=d["content"], source=d["source"])

async def tool_calculator(inp: CalculatorInput) -> CalculatorOutput:
    # Safe eval using ast.literal_eval for simple expressions
    # For production, use asteval or a restricted parser
    try:
        result = eval(inp.expression, {"__builtins__": {}}, {})
        return CalculatorOutput(result=float(result))
    except Exception as e:
        raise ValueError(f"Invalid expression: {e}")

TOOLS = {
    "search": {"input": SearchInput, "output": SearchOutput, "func": tool_search, "timeout": 5.0},
    "retrieve_doc": {"input": RetrieveDocInput, "output": RetrieveDocOutput, "func": tool_retrieve_doc, "timeout": 5.0},
    "calculator": {"input": CalculatorInput, "output": CalculatorOutput, "func": tool_calculator, "timeout": 2.0},
}

async def call_tool_guarded(name: str, params: Dict[str, Any], request_id: str, thread_id: str) -> Dict[str, Any]:
    """Execute tool with validation, retries, and correlation IDs."""
    if name not in TOOLS:
        raise ValueError(f"Unknown tool: {name}")
    spec = TOOLS[name]
    In, Out, func, timeout = spec["input"], spec["output"], spec["func"], spec["timeout"]

    try:
        valid_in = In.model_validate(params)
    except ValidationError as ve:
        raise ValueError(f"Invalid input for {name}: {ve.errors()}")

    delays = [0.25, 0.5, 1.0]
    last_exc = None
    t0 = time.time()
    for attempt, delay in enumerate(delays, start=1):
        try:
            res = await asyncio.wait_for(func(valid_in), timeout=timeout)
            valid_out = Out.model_validate(res.model_dump())
            latency_ms = int((time.time() - t0) * 1000)
            record = {
                "tool": name,
                "input": valid_in.model_dump(),
                "output": valid_out.model_dump(),
                "latency_ms": latency_ms,
                "attempt": attempt,
                "request_id": request_id,
                "thread_id": thread_id,
            }
            logger.info("tool_call", extra={"trace": record})
            return record
        except Exception as e:
            last_exc = e
            await asyncio.sleep(delay)
    raise RuntimeError(f"Tool {name} failed after retries: {last_exc}")

---

### Step 4: LLM Setup

**Why:** Deterministic LLM config (temperature=0, seed, retries, timeout) ensures reproducible outputs.

In [ ]:
%%writefile app/llm.py
from langchain_openai import ChatOpenAI
from app.config import OPENAI_API_KEY, OPENAI_MODEL

llm = ChatOpenAI(
    model=OPENAI_MODEL,
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_kwargs={"seed": 42},  # Deterministic sampling
    max_retries=2,
    timeout=30,
)

---

### Step 5: Planner Node

**Why:** Validates plan structure at generation time to fail fast before execution.

In [ ]:
%%writefile app/planner.py
from langchain_core.prompts import ChatPromptTemplate
from app.llm import llm
from app.schemas import Plan, Memory, AgentState
from app.tools import TOOLS

planner_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a deterministic planner. Produce a JSON plan to solve the task using ONLY the allowed tools. "
     "No loops, max 5 steps, each step must list exactly one tool and params matching its schema. "
     "Allowed tools: {allowed_tools}. If no solution, create a one-step plan explaining inability."),
    ("system", "Thread summary: {summary}"),
    ("human", "{user_input}")
])

async def plan_node(state: AgentState) -> dict:
    """Generate and validate plan."""
    memory = Memory.model_validate(state.memory) if state.memory else Memory()
    allowed = list(TOOLS.keys())
    structured = llm.with_structured_output(Plan, strict=True)
    chain = planner_prompt | structured
    try:
        plan_model = await chain.ainvoke({
            "user_input": state.user_input,
            "allowed_tools": ", ".join(allowed),
            "summary": memory.summary,
        })
        plan = plan_model.model_dump()
        # Validate each step's params against tool input schema
        for step in plan["steps"]:
            tool_spec = TOOLS[step["tool"]]
            tool_spec["input"].model_validate(step["params"])
        return {"plan": plan, "step_index": 0, "results": []}
    except Exception as e:
        from app.schemas import ErrorEnvelope
        err = ErrorEnvelope(code="PLANNING_ERROR", message=str(e), details={}).model_dump()
        return {"error": err}

---

### Step 6: Executor Node

**Why:** Executes steps with correlation IDs, retries, and structured error capture.

In [ ]:
%%writefile app/executor.py
from app.schemas import Plan, ErrorEnvelope, AgentState
from app.tools import call_tool_guarded

async def execute_node(state: AgentState) -> dict:
    """Execute current step with guarded tool call."""
    plan = Plan.model_validate(state.plan)
    idx = state.step_index
    if idx >= len(plan.steps):
        return {}
    step = plan.steps[idx]
    request_id = state.get("request_id", "")
    thread_id = state.thread_id

    try:
        record = await call_tool_guarded(step.tool, step.params, request_id, thread_id)
        return {"results": state.results + [record], "step_index": idx + 1}
    except Exception as e:
        err = ErrorEnvelope(
            code="TOOL_EXECUTION_ERROR",
            message=str(e),
            details={"tool": step.tool, "params": step.params, "step_index": idx},
            partial_results=state.results,
        ).model_dump()
        return {"error": err}

---

### Step 7: Finalizer Node

**Why:** Synthesizes final answer and updates memory for next turn.

In [ ]:
%%writefile app/finalizer.py
import time
from langchain_core.prompts import ChatPromptTemplate
from app.llm import llm
from app.schemas import FinalResult, Memory, Plan, AgentState
from app.tools import TOOLS

finalize_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a careful answerer. Use the provided tool results as evidence. "
               "Produce a concise, accurate final answer. Include a short plan summary and useful metadata."),
    ("system", "Allowed tools: {allowed_tools}"),
    ("human", "User input: {user_input}"),
    ("human", "Plan: {plan_json}"),
    ("human", "Results: {results_json}")
])

async def summarize_memory(summary: str, user_input: str, answer: str) -> str:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Summarize the conversation so far in ≤120 words, focusing on facts and decisions."),
        ("human", "Previous: {summary}"),
        ("human", "Last input: {user_input}"),
        ("human", "Last answer: {answer}")
    ])
    chain = prompt | llm
    out = await chain.ainvoke({"summary": summary, "user_input": user_input, "answer": answer})
    return out.content.strip()

async def finalize_node(state: AgentState) -> dict:
    """Synthesize final result and update memory."""
    try:
        allowed = ", ".join(TOOLS.keys())
        plan_json = Plan.model_validate(state.plan).model_dump()
        chain = finalize_prompt | llm.with_structured_output(FinalResult, strict=True)
        final = await chain.ainvoke({
            "allowed_tools": allowed,
            "user_input": state.user_input,
            "plan_json": plan_json,
            "results_json": state.results
        })
        final_result = final.model_dump()

        mem = Memory.model_validate(state.memory) if state.memory else Memory()
        new_summary = await summarize_memory(mem.summary, state.user_input, final_result["answer"])
        updated_mem = Memory(summary=new_summary, last_updated_ts=time.time()).model_dump()

        return {"final_result": final_result, "memory": updated_mem}
    except Exception as e:
        from app.schemas import ErrorEnvelope
        err = ErrorEnvelope(code="FINALIZATION_ERROR", message=str(e), details={}).model_dump()
        return {"error": err}

---

### Step 8: Error Node

**Why:** Captures all failure paths as structured `ErrorEnvelope`.

In [ ]:
%%writefile app/error_node.py
from app.schemas import AgentState

async def error_node(state: AgentState) -> dict:
    """Terminal error state."""
    return {}

---

### Step 9: Build the Graph

**Why:** LangGraph compiles the state machine with checkpoints for crash recovery.

In [ ]:
%%writefile app/graph.py
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from app.schemas import AgentState, Plan
from app.planner import plan_node
from app.executor import execute_node
from app.finalizer import finalize_node
from app.error_node import error_node

def route_after_plan(state: AgentState):
    return "execute" if state.plan and not state.error else "error"

def route_after_execute(state: AgentState):
    if state.error:
        return "error"
    plan = Plan.model_validate(state.plan)
    if state.step_index < len(plan.steps):
        return "execute"
    return "finalize"

builder = StateGraph(AgentState)
builder.add_node("plan", plan_node)
builder.add_node("execute", execute_node)
builder.add_node("finalize", finalize_node)
builder.add_node("error", error_node)

builder.set_entry_point("plan")
builder.add_conditional_edges("plan", route_after_plan, {"execute": "execute", "error": "error"})
builder.add_conditional_edges("execute", route_after_execute, {"execute": "execute", "finalize": "finalize", "error": "error"})
builder.add_edge("finalize", END)
builder.add_edge("error", END)

checkpointer = SqliteSaver.from_conn_string("checkpoints.sqlite")
graph = builder.compile(checkpointer=checkpointer)

---

### Step 10: FastAPI Endpoint

**Why:** Typed request/response models + OpenAPI schema for client generation.

In [ ]:
%%writefile app/main.py
import uuid
import logging
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel, StrictStr
from app.graph import graph
from app.schemas import AgentState, FinalResult, ErrorEnvelope

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
app = FastAPI(title="Deterministic LangGraph Agent")

class AgentRequest(BaseModel):
    thread_id: StrictStr
    user_input: StrictStr

@app.post("/agent", response_model=None)
async def agent_endpoint(req: AgentRequest):
    request_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": req.thread_id}}
    try:
        # Check if resuming from checkpoint
        existing_state = graph.get_state(config)
        resumed = bool(existing_state and existing_state.values)

        result: AgentState = await graph.ainvoke(
            {"thread_id": req.thread_id, "user_input": req.user_input, "request_id": request_id},
            config=config,
        )
        if result.final_result:
            payload = {
                "request_id": request_id,
                "thread_id": req.thread_id,
                "final_result": result.final_result,
                "metadata": {"resumed": resumed}
            }
            return JSONResponse(status_code=200, content=payload)
        if result.error:
            payload = {
                "request_id": request_id,
                "thread_id": req.thread_id,
                "error": result.error
            }
            return JSONResponse(status_code=422, content=payload)
        raise HTTPException(status_code=500, detail="Agent returned no result")
    except HTTPException:
        raise
    except Exception as e:
        payload = {
            "request_id": request_id,
            "thread_id": req.thread_id,
            "error": {"code": "UNHANDLED_ERROR", "message": str(e), "details": {}}
        }
        return JSONResponse(status_code=500, content=payload)

---

## Run and Validate

### Start the Server

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
from app.main import app

# Run in background (Colab)
import threading
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

import time
time.sleep(3)  # Wait for server to start
print("Server running at http://localhost:8000")

### Test the Endpoint

In [ ]:
import requests

url = "http://localhost:8000/agent"
payload = {
    "thread_id": "test-thread-1",
    "user_input": "What is LangGraph and calculate 2+2?"
}

response = requests.post(url, json=payload)
print(response.status_code)
print(response.json())

**Expected output:**

```json
{
  "request_id": "...",
  "thread_id": "test-thread-1",
  "final_result": {
    "answer": "LangGraph is a framework for building stateful, deterministic agents with graphs and checkpointers. 2+2 equals 4.",
    "evidence": [...],
    "plan_summary": "...",
    "metadata": {}
  },
  "metadata": {"resumed": false}
}
```

### Test Resume-on-Failure

Simulate a crash by killing the server mid-execution, then restart and re-send the same `thread_id`. The agent resumes from the last checkpoint.

In [ ]:
# Restart server and re-run the same request
response = requests.post(url, json=payload)
print(response.json()["metadata"]["resumed"])  # Should be True

---

## Next Steps

1. **Add real tools:** Integrate vector search, APIs, or databases
2. **Upgrade checkpointer:** Use Postgres for multi-worker production
3. **Add auth & rate limiting:** Protect `/agent` with API keys and quotas
4. **Observability:** Integrate LangSmith tracing and structured JSON logs
5. **Containerize:** Package with Docker for cloud deployment

You now have a deterministic, resumable agent that validates every I/O, logs every step, and recovers from crashes—ready to scale.